# Commodity Price Prediction
---
## Overview
Seiring dengan meningkatnya volatilitas pasar pangan global, memprediksi harga komoditas pangan dengan akurat menjadi tantangan penting bagi pembuat kebijakan, pelaku bisnis, dan peneliti. Kompetisi Prediksi Harga Komoditas Pangan ini memberikan kesempatan bagi peserta untuk menganalisis faktor-faktor yang mempengaruhi fluktuasi harga bahan pangan. Dataset yang digunakan mencakup tren harga historis, kondisi pasar, dan indikator ekonomi yang berperan dalam perubahan harga pangan di berbagai wilayah. Dengan menerapkan teknik data science, peserta dapat menggali wawasan mendalam tentang dinamika pasar yang mendasari pergerakan harga.

## Permasalahan
Fluktuasi harga komoditas pangan memiliki dampak ekonomi dan sosial yang signifikan, mulai dari anggaran rumah tangga hingga kebijakan perdagangan global. Dalam kompetisi ini, peserta ditantang untuk mengembangkan model prediktif yang dapat memperkirakan harga komoditas pangan berdasarkan data historis. Dataset yang diberikan mencakup berbagai fitur seperti harga sebelumnya, nilai tukar mata uang, tren pasokan global, serta indikator makroekonomi. Dengan menggunakan data tersebut, peserta diharapkan dapat membangun model yang mampu memprediksi harga setiap komoditas tiap tanggal pada rentang waktu tertentu dengan tingkat akurasi yang tinggi.

## Tujuan
Tujuan dari kompetisi ini adalah mendorong para data scientist untuk mengasah keterampilan mereka dalam membangun model prediksi yang akurat. Dengan berpartisipasi, peserta turut berkontribusi dalam pemahaman yang lebih mendalam mengenai pergerakan harga pangan dan dampaknya terhadap ketahanan pangan serta stabilitas ekonomi. Selain meningkatkan kemampuan teknis, kompetisi ini juga berfokus pada pentingnya pengambilan keputusan berbasis data dalam mengelola rantai pasokan pangan dan mengurangi risiko volatilitas harga.

# Importing Data and Libraries
---

In [ ]:
# Basics
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
# Preprocessing
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import MinMaxScaler
from datetime import timedelta
# Machine Learning Models
from xgboost import XGBRegressor, DMatrix
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn import svm
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
# Deep Learning
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
# Metrics
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, balanced_accuracy_score, accuracy_score
# Hyperparameter Tuning
import optuna
# Random State
RANDOM_STATE = 1805
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option("display.max_rows", 50)

## Train/Test

In [ ]:
train_path = '/kaggle/input/comodity-price-prediction-penyisihan-arkavidia-9/Harga Bahan Pangan/train'
test_path = '/kaggle/input/comodity-price-prediction-penyisihan-arkavidia-9/Harga Bahan Pangan/test'
filenames = os.listdir(train_path)

train = dict()
test = dict()
for file in filenames:
    print(file)
    train[file] = pd.read_csv(os.path.join(train_path,file))
    test[file] = pd.read_csv(os.path.join(test_path,file))
train

In [ ]:
# Define base path
base_path = '/kaggle/input/comodity-price-prediction-penyisihan-arkavidia-9/Google Trend'

google_trend_filenames = os.listdir(base_path)
google_trend_dfs = dict()

for folder in google_trend_filenames:
    commodity_df = None  
    
    folder_path = os.path.join(base_path, folder)
    for province_file in os.listdir(folder_path):
        province_path = os.path.join(folder_path, province_file)
        province_df = pd.read_csv(province_path, parse_dates=['Date'])
        
        # Extract province name from filename (remove .csv)
        province_name = province_file.rstrip('.csv')
        
        # Keep only relevant columns
        province_df = province_df[['Date', folder]].rename(columns={folder: province_name})
        
        # Merge with main DataFrame
        if commodity_df is None:
            commodity_df = province_df  # Initialize with first province data
        else:
            commodity_df = commodity_df.merge(province_df, on='Date', how='outer')  # Merge on Date
    
    # Store in dictionary 
    google_trend_dfs[folder] = commodity_df
    
    # Print results
    print('------------------------------------------------')
    print("GOOGLE TREND ON " + folder)
    print(commodity_df.isna().agg(lambda x : np.round(sum(x)/len(commodity_df) * 100,2)).sort_values(ascending=False))
    print('|||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||')
    print(commodity_df)  # Print only the first few rows


## Global Commodity

In [ ]:
commodity_path = '/kaggle/input/comodity-price-prediction-penyisihan-arkavidia-9/Global Commodity Price'
commodity_filenames = os.listdir(commodity_path)

commodity = dict()
for file in commodity_filenames:
    key_name = file.replace(' Historical Data.csv','')
    print(key_name)
    commodity[key_name] = pd.read_csv(os.path.join(commodity_path,file))
    commodity[key_name]['Date'] = pd.to_datetime(commodity[key_name]['Date'])
    commodity[key_name] = commodity[key_name].rename(columns={'Price': f"commodity_price"})
    numerical_columns = ['Open','High','Low', 'commodity_price']
    for nums in numerical_columns:
        commodity[key_name][nums] =commodity[key_name][nums].astype(str).str.replace(",", "").astype(float) 
    print(commodity[key_name].isna().agg(lambda x : np.round(sum(x)/len(commodity[key_name]) * 100,2)).sort_values(ascending=False))
    print(commodity[key_name].info())

## Currency

In [ ]:
uang_path = '/kaggle/input/comodity-price-prediction-penyisihan-arkavidia-9/Mata Uang'
filenames = os.listdir(uang_path)

uang = dict()
for file in filenames:
    uang[file[:6]] = pd.read_csv(os.path.join(uang_path,file))
    uang[file[:6]]['Date'] = pd.to_datetime(uang[file[:6]]['Date'])
    print(file[:6])
    print(uang[file[:6]].isna().agg(lambda x : np.round(sum(x)/len(uang[file[:6]]) * 100,2)).sort_values(ascending=False))
    print(uang[file[:6]].info())

# Data Description
---

## Whole Dataset Shape
| **Train Folder**                        | **Google Trends**                     | **Global Commodity Price**       | **Mata Uang**   |
|-----------------------------------------|---------------------------------------|----------------------------------|-----------------|
| Daging Ayam Ras.csv                     |  daging ayam                          | crude oil futures                | MYRUSD          |
| Daging Sapi Murni.csv                   |     daging sapi                    | natural gas futures              | SGDUSD          |
| Telur Ayam Ras.csv                      |      telur ayam                           | newcastle coal futures           | THBUSD          |
| Cabai Rawit Merah.csv                   | cabai rawit                                | palm oil futures                 | USDIDR          |
| Gula Konsumsi.csv                       | bawang putih                          | us sugar 11 futures              |                 |
| Beras Premium.csv                       | tepung terigu                         | us wheat futures                 |                 |
| Tepung Terigu (Curah).csv               |  tepung                              |                                  |                 |
| Beras Medium.csv                        |  beras                          |                                  |                 |
| Minyak Goreng Curah.csv                 |  bawang                           |                                  |                 |
| Bawang Merah.csv                        | cabai merah                           |                                  |                 |
| Minyak Goreng Kemasan Sederhana.csv     | minyak goreng                         |                                  |                 |
| Bawang Putih Bonggol.csv                | daging                                |                                  |                 |
| Cabai Merah Keriting.csv                | cabai                                 |                                  |                 |
|                                         | gula                                  |                                  |                 |
|                                         | bawang merah                          |                                  |                 |

## Train

In [ ]:
for key, data in train.items():
    print('--------------------------------------------')
    print(key)
    print("Start Date: ", min(data['Date']))
    print("End Date: ", max(data['Date']))

In [ ]:
for key, data in train.items():
    print("---------------------------------------------------------------")
    print("DESCRIBING "+key)
    print(data.describe())

In [ ]:
for key, data in train.items():
    print("---------------------------------------------------------------")
    print("NULL PERCENTAGE ON "+key)
    print(data.isna().agg(lambda x : np.round(sum(x)/len(data) * 100,2)).sort_values(ascending=False))

## Test

In [ ]:
for key, data in test.items():
    print('--------------------------------------------')
    print(key)
    print("Start Date: ", min(data['Date']))
    print("End Date: ", max(data['Date']))
    break

## Google Trend

In [ ]:
for key, data in google_trend_dfs.items():
    print('--------------------------------------------')
    print(key)
    print("Start Date: ", min(data['Date']))
    print("End Date: ", max(data['Date']))
    break

## Global Commodity Prices

In [ ]:
for key, data in commodity.items():
    print('--------------------------------------------')
    print(key)
    print("Start Date: ", min(data['Date']))
    print("End Date: ", max(data['Date']))
    break

## Currency

In [ ]:
for key, data in uang.items():
    print('--------------------------------------------')
    print(key)
    print("Start Date: ", min(data['Date']))
    print("End Date: ", max(data['Date']))
    break

# Data Preprocessing
---

## ChatGPT Recommendations


| **Commodity (Train Folder)** | **Google Trends (Demand Indicator)** | **Global Commodity Prices (Macroeconomic Factors)** | **Currency Exchange Rates (Import/Export Influence)** |
|-----------------------------|--------------------------------------|-----------------------------------------------------|-----------------------------------------------------|
| **Daging Ayam Ras** (Chicken Meat) | daging ayam, daging | None (but feed prices could be a factor) | USDIDR (feed imports), THBUSD (regional suppliers) |
| **Daging Sapi Murni** (Beef) | daging sapi, daging | None | USDIDR (imported beef), MYRUSD, THBUSD (regional suppliers) |
| **Telur Ayam Ras** (Chicken Eggs) | telur ayam | None | USDIDR (feed imports) |
| **Cabai Rawit Merah** (Red Bird's Eye Chili) | cabai rawit, cabai merah, cabai | None | USDIDR (fertilizer imports) |
| **Cabai Merah Keriting** (Curly Red Chili) | cabai merah, cabai rawit, cabai | None | USDIDR (fertilizer imports) |
| **Gula Konsumsi** (Sugar) | gula | **US Sugar #11 Futures** | USDIDR (imported sugar) |
| **Beras Premium** (Premium Rice) | beras | None (Rice is mostly domestically produced) | USDIDR (some imported rice), THBUSD (Thai rice market) |
| **Beras Medium** (Medium Rice) | beras | None | USDIDR (some imported rice), THBUSD |
| **Tepung Terigu (Curah)** (Wheat Flour) | tepung, tepung terigu | **US Wheat Futures** | USDIDR (imported wheat), MYRUSD (regional wheat suppliers) |
| **Minyak Goreng Curah** (Bulk Cooking Oil) | minyak goreng | **Palm Oil Futures** | USDIDR, MYRUSD (Malaysia = major palm oil exporter) |
| **Minyak Goreng Kemasan Sederhana** (Packaged Cooking Oil) | minyak goreng | **Palm Oil Futures** | USDIDR, MYRUSD |
| **Bawang Merah** (Shallots) | bawang merah, bawang | None | USDIDR (imported fertilizers) |
| **Bawang Putih Bonggol** (Garlic) | bawang putih, bawang | None | USDIDR (imported garlic, mostly from China) |

In [ ]:
commodity_features = {
    "Daging Ayam Ras.csv": {
        "google_trends": ["daging ayam", "daging"],
        "global_commodity_prices": [],
        "currency_exchange": ["USDIDR", "THBUSD"]
    },
    "Daging Sapi Murni.csv": {
        "google_trends": ["daging sapi", "daging"],
        "global_commodity_prices": [],
        "currency_exchange": ["USDIDR", "MYRUSD", "THBUSD"]
    },
    "Telur Ayam Ras.csv": {
        "google_trends": ["telur ayam"],
        "global_commodity_prices": [],
        "currency_exchange": ["USDIDR"]
    },
    "Cabai Rawit Merah.csv": {
        "google_trends": ["cabai rawit", "cabai merah", "cabai"],
        "global_commodity_prices": [],
        "currency_exchange": ["USDIDR"]
    },
    "Cabai Merah Keriting.csv": {
        "google_trends": ["cabai merah", "cabai rawit", "cabai"],
        "global_commodity_prices": [],
        "currency_exchange": ["USDIDR"]
    },
    "Gula Konsumsi.csv": {
        "google_trends": ["gula"],
        "global_commodity_prices": ["US Sugar 11 Futures"],
        "currency_exchange": ["USDIDR"]
    },
    "Beras Premium.csv": {
        "google_trends": ["beras"],
        "global_commodity_prices": [],
        "currency_exchange": ["USDIDR", "THBUSD"]
    },
    "Beras Medium.csv": {
        "google_trends": ["beras"],
        "global_commodity_prices": [],
        "currency_exchange": ["USDIDR", "THBUSD"]
    },
    "Tepung Terigu (Curah).csv": {
        "google_trends": ["tepung", "tepung terigu"],
        "global_commodity_prices": ["US Wheat Futures"],
        "currency_exchange": ["USDIDR", "MYRUSD"]
    },
    "Minyak Goreng Curah.csv": {
        "google_trends": ["minyak goreng"],
        "global_commodity_prices": ["Palm Oil Futures"],
        "currency_exchange": ["USDIDR", "MYRUSD"]
    },
    "Minyak Goreng Kemasan Sederhana.csv": {
        "google_trends": ["minyak goreng"],
        "global_commodity_prices": ["Palm Oil Futures"],
        "currency_exchange": ["USDIDR", "MYRUSD"]
    },
    "Bawang Merah.csv": {
        "google_trends": ["bawang merah", "bawang"],
        "global_commodity_prices": [],
        "currency_exchange": ["USDIDR"]
    },
    "Bawang Putih Bonggol.csv": {
        "google_trends": ["bawang putih", "bawang"],
        "global_commodity_prices": [],
        "currency_exchange": ["USDIDR"]
    }
}

## Pivot Tables and Null Treatment

Steps:
- Turn columns into row values (province)
- Interpolate missing data

In [ ]:
train_ready = dict()
for key, dataframes in train.items():
    # Convert Date column to datetime format
    dataframes["Date"] = pd.to_datetime(dataframes["Date"])
    
    # Melt the data to long format
    dataframes_long = dataframes.melt(id_vars=["Date"], var_name="Region", value_name="Price")

    # Interpolate missing price values
    dataframes_long["Price"] = dataframes_long["Price"].interpolate(method="linear")
    dataframes_long["Price"] = dataframes_long["Price"].bfill()
    
    # Check for missing values
    missing_values = dataframes_long.isna().sum()
    
    # Display the transformed data and missing values
    print('--------------------------------------------')
    print("NULL VALUES on", key)
    print(missing_values)
    print(dataframes_long.head())
    train_ready[key] = dataframes_long

In [ ]:
google_trend_dfs_ready = dict()
for key, dataframes in google_trend_dfs.items():
    # Convert Date column to datetime format
    dataframes["Date"] = pd.to_datetime(dataframes["Date"])
    
    # Melt the data to long format
    dataframes_long = dataframes.melt(id_vars=["Date"], var_name="Region", value_name="Searches")

    # Interpolate missing price values
    dataframes_long["Searches"] = dataframes_long["Searches"].interpolate(method="linear")
    dataframes_long["Searches"] = dataframes_long["Searches"].bfill()
    
    # Check for missing values
    missing_values = dataframes_long.isna().sum()
    
    # Display the transformed data and missing values
    print('--------------------------------------------')
    print("NULL VALUES on", key)
    print(missing_values)
    print(dataframes_long.head())
    
    google_trend_dfs_ready[key] = dataframes_long

# EDA
---

## Latest Commodity Prices per Province

In [ ]:
i = 0
sns.set_style('whitegrid')
f, ax = plt.subplots(nrows=5, ncols=3, figsize=(25, 40))

for commodity, data in train_ready.items():
    latest_price = data.groupby('Region').last()
    latest_price = latest_price.reset_index()
    latest_price = latest_price[['Region', 'Price']]
    latest_price = latest_price.sort_values(by='Price', ascending=False)
    ax[i//3, i%3].set_title('Price of ' + commodity)
    ax[i//3, i%3].xaxis.set_tick_params(rotation=40)
    sns.barplot(y='Region', x='Price', data=latest_price, ax=ax[i//3, i%3])
    i+=1

## Latest Median Prices per Commodities

In [ ]:
median_prices = dict()
for commodity, data in train_ready.items():
    median_prices[commodity] = data['Price'].median()
median_prices = pd.DataFrame(median_prices.items(), columns=['Commodity', 'Price'])
median_prices = median_prices.sort_values(by='Price', ascending=False)
plt.figure(figsize=(10, 5))
plt.title('Median prices of 30-09-2024')
sns.barplot(y='Commodity', x='Price', data=median_prices)
plt.xticks(rotation=90)
plt.show()

## TimeSeries Plot (Median)

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Function to calculate EMA
def calculate_ema(series, span=30):
    return series.ewm(span=span, adjust=False).mean()

sns.set_style('whitegrid')
f, ax = plt.subplots(nrows=5, ncols=3, figsize=(15, 25))
ax = ax.flatten()  # Flatten for easier indexing

i = 0
for commodity, data in train_ready.items():
    latest_price = data.groupby('Date', as_index=False)['Price'].median()
    latest_price['EMA'] = calculate_ema(latest_price['Price'], span=30)  # Ensure consistency

    # Handle Inf values
    latest_price.replace([np.inf, -np.inf], np.nan, inplace=True)
    latest_price.dropna(inplace=True)

    ax[i].set_title('Price of ' + commodity)
    
    # Plot Actual Prices
    sns.lineplot(x='Date', y='Price', data=latest_price, label='Actual', linewidth=2, ax=ax[i])

    # Plot EMA
    sns.lineplot(x='Date', y='EMA', data=latest_price, label='EMA (30-day)', 
                 linewidth=2, linestyle='dashed', color='red', ax=ax[i])

    i += 1
    if i >= len(ax):  # Stop if there are more commodities than subplots
        break

plt.tight_layout()
plt.show()

# Feature Prediction for Testing
---
## Mata Uang

In [ ]:
uang_modified = dict()
for key, df in uang.items():
    # Convert Date column to datetime and sort
    df["Date"] = pd.to_datetime(df["Date"])
    df = df.sort_values(by="Date")
    
    # Create moving averages
    df["Close H+1"] = df["Close"].shift(periods= -1)
    df["MA7"] = df["Close"].rolling(window=7).mean()
    df["MA30"] = df["Close"].rolling(window=30).mean()
    # Create rolling standard deviation (volatility)
    df["Volatility7"] = df["Close"].rolling(window=7).std()

    # Extract time-based features
    df["DayOfWeek"] = df["Date"].dt.weekday
    # df["Week"] = df["Date"].dt.isocalendar().week
    df["Month"] = df["Date"].dt.month
    # df["Quarter"] = df["Date"].dt.quarter
    # df["Year"] = df["Date"].dt.year

    df.drop(columns=["Adj Close", "Volume", "Date", "High", "Low", "Open"], inplace=True)
    
    # Drop NaN values caused by shifting and rolling operations
    df.dropna(inplace=True)
    uang_modified[key] = df.copy()
    print("---------------------------------------")
    print(key)
    print(uang_modified[key].head(3))

In [ ]:
# Function to make the sequences
def create_sequences(data, seq_length=10):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i + seq_length].drop(columns=["Close H+1"]))
        y.append(data[i:i + seq_length]["Close H+1"])
    return np.array(X), np.array(y)

# Function to update sequence
def update_sequence(last_sequence, pred, col_indices):
    global last_known_date
    last_known_date += pd.Timedelta(days=1)
    new_row = last_sequence[1:].copy()
    new_row = np.append(new_row, last_sequence[-1:].copy(), axis=0)
    
    new_row[0,-1, col_indices["Close"]] = pred
    new_row[0,-1, col_indices["MA7"]] = np.mean(last_sequence[0, -7:, col_indices["Close"]])
    new_row[0,-1, col_indices["MA30"]] = np.mean(last_sequence[0, -30:, col_indices["Close"]])
    new_row[0,-1, col_indices["Volatility7"]] = np.std(last_sequence[0, -7:, col_indices["Close"]])    
    new_row[0,-1,col_indices["DayOfWeek"]] = last_known_date.dayofweek
    new_row[0,-1,col_indices["Month"]] = last_known_date.month
    # new_row[0,-1,col_indices["Quarter"]] = last_known_date.quarter
    # new_row[0,-1,col_indices["Year"]] = last_known_date.year

    return new_row

In [ ]:
uang_predicted = dict()

for key, df in uang_modified.items():
    columns = list(df.columns)
    panjang_sequence = 30
    
    train_size_uang = int(len(df)*0.8)
    train_uang = df[:train_size_uang]
    test_uang = df[train_size_uang:]
    
    scaler = MinMaxScaler()
    train_scaled_uang = pd.DataFrame(scaler.fit_transform(train_uang), columns=columns)
    test_scaled_uang = pd.DataFrame(scaler.transform(test_uang), columns=columns)

    X_train_uang, y_train_uang = create_sequences(train_scaled_uang, panjang_sequence)
    X_test_uang, y_test_uang = create_sequences(test_scaled_uang, panjang_sequence)
    
    # Define the LSTM model
    jumlah_fitur = X_train_uang.shape[2]
    input_layer = tf.keras.Input(shape=(panjang_sequence, jumlah_fitur))
    lstm = tf.keras.layers.LSTM(50, return_sequences=True)(input_layer)
    lstm = tf.keras.layers.Dropout(0.2, seed=RANDOM_STATE)(lstm)
    lstm = tf.keras.layers.LSTM(25, return_sequences=False)(lstm)
    lstm = tf.keras.layers.Dense(10, activation='relu')(lstm)
    output_layer = tf.keras.layers.Dense(1)(lstm)

    # Build the Model
    model = tf.keras.Model(inputs=input_layer, outputs=output_layer)
    model.compile(
        optimizer='adam', 
        loss='mse', 
        metrics=['mse'])

    early_stop = tf.keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=5,
        restore_best_weights=True
    )
    history = model.fit(X_train_uang, y_train_uang,
                        batch_size=16,
                        epochs=100,
                        validation_data=(X_test_uang, y_test_uang),
                        callbacks=[early_stop], verbose=0)

    # Prediksi 92 hari ke depan
    jumlah_hari_forecast = 92
    last_sequence = np.array(test_scaled_uang[-panjang_sequence:].drop(columns=["Close"])).reshape(1, panjang_sequence, -1)
    last_known_date = pd.Timestamp("2024-09-30")
    col_indices = {
        "Close": 0,
        "MA7": 1,
        "MA30": 2,
        "Volatility7": 3,
        "DayOfWeek": 4,
        "Month": 5
    }
    predictions = []

    for _ in range(jumlah_hari_forecast):
        pred = model.predict(last_sequence)[0,0]
        predictions.append(pred)
        
        # Update the sequence
        last_sequence = update_sequence(last_sequence, pred, col_indices)

    predictions_real = np.array(predictions, dtype=np.float64) * scaler.data_range_[col_indices["Close"]] + scaler.data_min_[col_indices["Close"]]
    
    # Create future dates
    future_dates = pd.date_range(start="2024-10-01", periods=jumlah_hari_forecast, freq="D")
    
    # Save predictions
    uang_predicted[key] = pd.DataFrame({"Date": future_dates, "Predicted_Close": predictions_real})

## Komoditas

In [ ]:
commodity_copy = dict()
for key, df in commodity.items():
    df["Date"] = pd.to_datetime(df["Date"])
    df = df.sort_values(by="Date")
    df.rename(columns={"commodity_price": "Close"}, inplace=True)
    df["Close"] = df["Close"].astype(str).str.replace(",", "").astype(float)
    df["CloseH+1"] = df["Close"].shift(periods=1)
    
    # Create moving averages
    df["MA7"] = df["Close"].rolling(window=7).mean()
    df["MA30"] = df["Close"].rolling(window=30).mean()
    
    # Create rolling standard deviation (volatility)
    df["Volatility7"] = df["Close"].rolling(window=7).std()

    # Extract time-based features
    df["DayOfWeek"] = df["Date"].dt.weekday
    df["Month"] = df["Date"].dt.month
    # df["Quarter"] = df["Date"].dt.quarter
    # df["Year"] = df["Date"].dt.year

    # Drop unnecessary columns if they exist
    columns_to_drop = ["Vol.", "Change %", "Date", "Open", "High", "Low"]
    df.drop(columns=[col for col in columns_to_drop if col in df.columns], inplace=True)

    # Drop NaN values caused by rolling operations
    df.dropna(inplace=True)

    # Store the processed dataframe in a new dictionary
    commodity_copy[key] = df.copy()

    # Display processed data for debugging
    print('===================================================================')
    print(f"Processed data for {key}")
    print(df.info())
    print(df.head(3))

In [ ]:
# Function to make sequences with proper handling of missing values
def create_sequences(data, seq_length=10):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data.iloc[i:i + seq_length].drop(columns=["CloseH+1"]).values)
        y.append(data.iloc[i + seq_length]["CloseH+1"])
    
    return np.array(X), np.array(y)

# Function to update sequence dynamically with new predictions
def update_sequence(last_sequence, pred, col_indices):
    new_row = np.copy(last_sequence)

    # Shift left
    new_row[0, :-1, :] = new_row[0, 1:, :]  
    new_row[0, -1, col_indices["Close"]] = pred  

    # Only update if the column exists
    for col in ["DayOfWeek", "Month"]:
        if col in col_indices and col_indices[col] < new_row.shape[2]:  
            new_row[0, -1, col_indices[col]] = getattr(last_known_date, col.lower())
    
    new_row[0,-1, col_indices["MA7"]] = np.mean(last_sequence[0, -7:, col_indices["Close"]])
    new_row[0,-1, col_indices["MA30"]] = np.mean(last_sequence[0, -30:, col_indices["Close"]])
    new_row[0,-1, col_indices["Volatility7"]] = np.std(last_sequence[0, -7:, col_indices["Close"]])    
    return new_row

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
commodity_price_predicted = {}

# Iterate through each commodity dataset
for key, df in commodity_copy.items():
    columns = list(df.columns)
    sequence_lenght = 30

    # Train-test split (80-20)
    train_size_commodity = int(len(df) * 0.8)
    train_commodity = df[:train_size_commodity]
    test_commodity = df[train_size_commodity:]

    for col in df.columns:
        df[col] = df[col].astype(str).str.replace(',', '').astype(float)

        # Train-test split (80-20)
        train_size_commodity = int(len(df) * 0.8)
        train_commodity = df[:train_size_commodity]
        test_commodity = df[train_size_commodity:]

    # Apply MinMaxScaler
    scaler = MinMaxScaler()
    train_scaled_commodity = pd.DataFrame(scaler.fit_transform(train_commodity), columns=df.columns)
    test_scaled_commodity = pd.DataFrame(scaler.transform(test_commodity), columns=df.columns)


    # Feature Scaling
    columns = list(df.columns)
    scaler = MinMaxScaler()
    train_scaled_commodity = pd.DataFrame(scaler.fit_transform(train_commodity), columns=columns)
    test_scaled_commodity = pd.DataFrame(scaler.transform(test_commodity), columns=columns)


    # Create sequences
    X_train_commodity, y_train_commodity = create_sequences(train_scaled_commodity, sequence_lenght)
    X_test_commodity, y_test_commodity = create_sequences(test_scaled_commodity, sequence_lenght)

    # Define the LSTM model
    jumlah_fitur = X_train_commodity.shape[2]
    input_layer = tf.keras.Input(shape=(sequence_lenght, jumlah_fitur))
    lstm = tf.keras.layers.LSTM(50, return_sequences=True)(input_layer)
    lstm = tf.keras.layers.Dropout(0.2)(lstm)
    lstm = tf.keras.layers.LSTM(25, return_sequences=False)(lstm)
    lstm = tf.keras.layers.Dense(10, activation='relu')(lstm)
    output_layer = tf.keras.layers.Dense(1)(lstm)

    # Compile the model
    model = tf.keras.Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer='adam', loss='mse', metrics=['mse'])

    # Early stopping to prevent overfitting
    early_stop = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

    print("Fitting model...")
    # Train the model
    history = model.fit(X_train_commodity, y_train_commodity,
                        batch_size=16,
                        epochs=100,
                        validation_data=(X_test_commodity, y_test_commodity),
                        callbacks=[early_stop], verbose=0)

    # Predict next 92 days
    jumlah_hari_forecast = 92
    last_sequence = np.array(test_scaled_commodity[-sequence_lenght:].drop(columns=["Close"])).reshape(1, sequence_lenght, -1)
    last_known_date = pd.Timestamp("2024-10-01")

    col_indices = {col: idx for idx, col in enumerate(columns)}
    predictions = []
    
    print("Predicting model...")
    for _ in range(jumlah_hari_forecast):
        pred = model.predict(last_sequence, verbose=0)[0, 0]
        predictions.append(pred)
        last_sequence = update_sequence(last_sequence, pred, col_indices)

    # Convert predictions back to original scale
    predictions_real = np.array(predictions) * scaler.data_range_[col_indices["Close"]] + scaler.data_min_[col_indices["Close"]]

    # Create future dates
    future_dates = pd.date_range(start="2024-09-30", periods=jumlah_hari_forecast, freq="D")

    # Store predictions
    commodity_price_predicted[key] = pd.DataFrame({"Date": future_dates, "commodity_price": predictions_real})

# Modelling
---

## Modelling Preprocessing

In [ ]:
def create_features(df_input, feats):
    df = df_input.copy()
    df['lag_7'] = df.groupby('Region')['Price'].shift(7).bfill()
    df['lag_30'] = df.groupby('Region')['Price'].shift(30).bfill()
    df['rolling_7'] = df.groupby('Region')['Price'].transform(lambda x: x.rolling(7, min_periods=1).mean())
    df['rolling_30'] = df.groupby('Region')['Price'].transform(lambda x: x.rolling(30, min_periods=1).mean())
    
    # Time-based features
    df['month'] = df['Date'].dt.month
    df['weekday'] = df['Date'].dt.weekday

    # Features
    global_comm_feats = feats['global_commodity_prices'] 
    currency_feats = feats['currency_exchange']

    for col in global_comm_feats:
        print("merging ", col)
        df = df.merge(commodity[col][["Date", "commodity_price"]], on="Date", how="left")
        df.rename(columns={'commodity_price': f'commodity_price_{col}'},inplace=True)
        df[f'commodity_price_{col}'] = df[f'commodity_price_{col}'].ffill()
        df[f'commodity_price_{col}'] = df[f'commodity_price_{col}'].bfill()
        
    for col in currency_feats:
        print("merging ", col)
        df = df.merge(uang[col][["Date", "Adj Close"]], on="Date", how="left")
        df.rename(columns={'Adj Close': f'currency_{col}'},inplace=True)
        df[f'currency_{col}'] = df[f'currency_{col}'].ffill()
        df[f'currency_{col}'] = df[f'currency_{col}'].bfill()

    return df

In [ ]:
def forecast_next_92_days(df, model, target, last_date, commodity_type, region):
    """
    Forecast the next 92 days from latest row of data
    """
    predictions = []
    last_known_data = df.iloc[-60:].copy()  # Take last 60 days for lag features

    for i in range(92):
        next_date = last_date + timedelta(days=i + 1)
        new_row = last_known_data.iloc[-1:].copy()  # Use last row as base

        # Update rolling features dynamically
        new_row['lag_7'] = last_known_data[target].iloc[-7]
        new_row['lag_30'] = last_known_data[target].iloc[-30]
        new_row['rolling_7'] = last_known_data[target].iloc[-7:].mean()
        new_row['rolling_30'] = last_known_data[target].iloc[-30:].mean()

        # Time features
        new_row['month'] = next_date.month
        new_row['weekday'] = next_date.weekday()

        # Corrected currency exchange feature lookup
        for col in commodity_features[commodity_type]['currency_exchange']:
            if last_date <= pd.to_datetime('2024-09-30'):
                matching_row = uang[col].loc[uang[col]['Date'] == last_date, 'Close']
            else:
                matching_row = uang_predicted[col].loc[uang_predicted[col]['Date'] == last_date, 'Predicted_Close']
            new_row[f'currency_{col}'] = matching_row.values[0]  # Extract single value
            
        # Corrected global commodity prices lookup
        for col in commodity_features[commodity_type]['global_commodity_prices']:
            if (last_date <= pd.to_datetime('2024-09-30')):
                matching_row = commodity[col].loc[commodity[col]['Date'] == last_date, 'commodity_price']
            else:    
                matching_row = commodity_price_predicted[col].loc[commodity_price_predicted[col]['Date'] == last_date, 'commodity_price']
            new_row[f'commodity_price_{col}'] = matching_row.values[0]
            
        
        # Drop Unnecessary Data
        new_row.drop(columns=[target, 'Region', 'Date'], inplace=True, errors='ignore')

        # Predict next day
        pred_price = model.predict(pd.DataFrame(new_row))[0]
        
        # Format the ID
        prediction_id = f"{commodity_type.replace('.csv','')}/{region}/{next_date.strftime('%Y-%m-%d')}"
        predictions.append((prediction_id, pred_price))

        # Append to last_known_data
        new_entry = pd.DataFrame({target: [pred_price]}, index=[next_date])
        last_known_data = pd.concat([last_known_data, new_entry])

    return predictions

In [ ]:
# Train and predict for each DataFrame in train_ready
target = 'Price'

def train_pipeline(base_model):
    """
    Forecast the next 92 days from latest row of data
    params:
    base_model = Model instantiator
    """
    predictions_dict = {}
    for key, df in train_ready.items():
        print(f"Processing {key}...")
        predictions_list = []
        df = create_features(df, commodity_features[key])
        for region in df['Region'].unique():
            print(f"Processing Region: {region}")
            
            # Define Region Data
            region_df = df[df['Region'] == region]
            X = region_df.drop([target]+['Region','Date'],axis=1)
            y = region_df[target]
            
            # Train model on full data
            model = base_model            
            model.fit(X, y)
    
            # Predict next 92 days
            last_date = df['Date'].max()
            pred_values = forecast_next_92_days(region_df, model, target, last_date, key, region)
    
            # Store predictions
            predictions_list.append(pd.DataFrame(pred_values, columns=['id', 'Predicted Price']))
    
        predictions_dict[key] = pd.concat(predictions_list, ignore_index=True)
    return predictions_dict

## XGBoost Hyperparameter
---

In [ ]:
# Prediction with Hyperparameter Tuning
predictions_dict = dict()
target = 'Price'
overall_mape_scores = []
for key, df in train_ready.items():
    print(f"Processing {key}...")
    predictions_list = []
    df = create_features(df, commodity_features[key])
    missing_values = df.isna().sum()
    print(missing_values)
    df.dropna(inplace=True)
    for region in df['Region'].unique():
        print(f"Processing Region: {region}")
        
        # Define Region Group
        region_df = df[df['Region'] == region]
        X = region_df.drop([target]+['Region','Date'],axis=1)
        y = region_df[target]

        tscv = TimeSeriesSplit(n_splits=3)
        
        # Set model parameters
        params = {
            'learning_rate': [0.01, 0.05, 0.1],
            'max_depth': [4, 6, 8],
            'n_estimators': [300, 600, 1000],
        }

        # Train model on full data
        model = XGBRegressor(random_state=RANDOM_STATE, tree_method='hist', objective='reg:squarederror')
        grid = GridSearchCV(model, param_grid=params,
                            scoring='neg_mean_absolute_percentage_error',
                            cv=tscv,
                            n_jobs=-1)
        grid.fit(X, y)

        best_params = grid.best_params_
        best_model = XGBRegressor(**best_params, random_state=RANDOM_STATE, tree_method='hist', objective='reg:squarederror')
        best_model.fit(X, y)

        # Predict next 92 days
        last_date = region_df['Date'].max()
        pred_values = forecast_next_92_days(region_df, best_model, target, last_date, key, region)

        # Store predictions
        predictions_list.append(pd.DataFrame(pred_values, columns=['id', 'Predicted Price']))

    predictions_dict[key] = pd.concat(predictions_list, ignore_index=True)

predictions = predictions_dict

## Random Forest Hyperparameter
---

In [ ]:
# TimeSeries Split CV test
predictions_df = {}
features = ['lag_7', 'lag_30', 'rolling_7', 'rolling_30', 'month', 'weekday']
target = 'Price'
overall_mape_scores = []

# Define the hyperparameter grid
param_dist = {
    'n_estimators': [100, 300, 500, 1000],
    'max_depth': [10, 15, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': [0.5, 0.7, 'sqrt', 'log2'],
    'bootstrap': [True, False]
}

predictions_dict = dict()
for key, df in train_ready.items():
    print(f"Processing {key}...")
    predictions_list = []
    df = create_features(df, commodity_features[key])
    
    # Handle missing values
    df.dropna(inplace=True)

    for region in df['Region'].unique():
        print(f"Processing Region: {region}")
        
        # Define Region Group
        region_df = df[df['Region'] == region]
        X = region_df.drop([target] + ['Region', 'Date'], axis=1)
        y = region_df[target]

        # TimeSeriesSplit Cross-Validation
        tscv = TimeSeriesSplit(n_splits=5)
        mape_scores = [] 

        # Run RandomizedSearchCV with TimeSeriesSplit
        rf = RandomForestRegressor(random_state=42, n_jobs=-1)
        random_search = RandomizedSearchCV(
            estimator=rf, 
            param_distributions=param_dist, 
            n_iter=10,  # Try 10 different random combinations
            cv=tscv, 
            verbose=1, 
            n_jobs=-1,
            scoring='neg_mean_absolute_percentage_error'
        )

        # Fit hyperparameter tuning on training data
        random_search.fit(X, y)

        # Get best parameters
        best_params = random_search.best_params_
        print(f"Best Parameters for {region}: {best_params}")

        # Train model with the best parameters
        best_model = RandomForestRegressor(**best_params, random_state=42, n_jobs=-1)
        best_model.fit(X, y)
    
        # Predict next 92 days
        last_date = df['Date'].max()
        pred_values = forecast_next_92_days(region_df, best_model, target, last_date, key, region)

        # Store predictions
        predictions_list.append(pd.DataFrame(pred_values, columns=['id', 'Predicted Price']))
    
    predictions_dict[key] = pd.concat(predictions_list, ignore_index=True)

predictions = predictions_dict

## ARIMA
---

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

def train_pipeline(target, arima_order):
    predictions_dict = {}
    for key, df in train_ready.items():
        print(f"Processing {key}...")
        predictions_list = []
        
        df = create_features(df, commodity_features[key])
        
        # Convert Date column to datetime and set it as the index
        df['Date'] = pd.to_datetime(df['Date'])
        df.set_index('Date', inplace=True)
        
        for region in df['Region'].unique():
            print(f"Processing Region: {region}")
            
            # Filter data for the region
            region_df = df[df['Region'] == region]
            y = region_df[target]
            
            # Train ARIMA model
            model = ARIMA(y, order=arima_order)  # Corrected ARIMA initialization
            model_fit = model.fit()
            
            # Predict next 92 days
            last_date = y.index.max()
            pred_values = model_fit.forecast(steps=92)  # Forecasting future prices
            
            # Create forecast dates
            pred_dates = pd.date_range(start=last_date + timedelta(days=1), periods=92, freq='D')
            
            # Format predictions
            pred_df = pd.DataFrame({
                'id': [f"{key}/{region}/{date.strftime('%Y-%m-%d')}" for date in pred_dates],
                'Predicted Price': pred_values
            })
            predictions_list.append(pred_df)
    
        predictions_dict[key] = pd.concat(predictions_list, ignore_index=True)
    
    return predictions_dict  # Return predictions & trained models

# Example usage with ARIMA order (p=5, d=1, q=0)
arima_order = (5, 1, 0)  # You can tune these values
predictions = train_pipeline(target='Price', arima_order=arima_order)

# Submission
---

In [ ]:
prediction_final = pd.DataFrame(columns=['id','Predicted Price'])
for key, df in predictions.items():
    print(df.head(3))
    prediction_final = pd.concat([prediction_final, df])
prediction_final = prediction_final.rename(columns={'Predicted Price':'price'})
print(prediction_final.head())
prediction_final.to_csv('submission.csv',index=False)